In [121]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Model

import numpy as np

In [158]:
def get_data():
    mnist = tf.keras.datasets.mnist
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    x_train, x_test = x_train / 255.0, x_test / 255.0
    y_train = tf.one_hot(y_train, 10)
    y_test = tf.one_hot(y_test, 10)

    train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000, reshuffle_each_iteration=True).batch(100)
    test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).shuffle(10000).batch(100)
    return train_ds, test_ds

train_ds, test_ds = get_data()

batch1 = train_ds.as_numpy_iterator().next()
batch1x = batch1[0]
batch1y = batch1[1]

In [208]:
model = keras.Sequential(
    [
        layers.Flatten(input_shape=(28,28)),
        layers.Dense(10, activation="linear"),
    ]
)
model.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_16 (Flatten)        (None, 784)               0         
                                                                 
 dense_23 (Dense)            (None, 10)                7850      
                                                                 
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________


In [197]:
model = keras.Sequential(
    [
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10)
    ]
)

In [204]:
# prediction example
prediction = model(batch1x[:1]) 
prediction_softmax = tf.nn.softmax(prediction)
print('Prediction (logit): ', np.round(prediction.numpy(), 2))
print('Prediction (softmax): ', np.round(prediction_softmax.numpy(), 2))

Prediction (logit):  [[ 2.02 -5.52 -3.43  1.07 -4.75 11.27 -1.85 -2.92  6.94  2.36]]
Prediction (softmax):  [[0.   0.   0.   0.   0.   0.99 0.   0.   0.01 0.  ]]


In [205]:
# define loss object
loss_object = keras.losses.CategoricalCrossentropy(from_logits=True)
loss = loss_object(batch1y[:1], prediction)

print("Loss: ", loss.numpy())

Loss:  0.0133690825


In [206]:
# define loss and accuracy metric for displaying for human user 
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')

print("Loss: ", train_loss(loss).numpy())
print("Accuracy: ", train_accuracy(batch1y[0], prediction).numpy())

Loss:  0.0133690825
Accuracy:  1.0


In [203]:
optimizer = keras.optimizers.experimental.SGD(learning_rate=0.5)
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])
model.fit(train_ds, epochs=3)

Epoch 1/3
600/600 [==============================] - 1s 1ms/step - loss: 0.3093 - accuracy: 0.9047
Epoch 2/3
600/600 [==============================] - 1s 1ms/step - loss: 0.1504 - accuracy: 0.9553
Epoch 3/3
600/600 [==============================] - 1s 1ms/step - loss: 0.1196 - accuracy: 0.9637
